In [4]:
# An attempt to recreate studies/cuda-kernels/reducers/awkward_reduce_sum_atomics.py using cccl instead of raw cuda kernels
import awkward as ak
import cupy as cp
import numpy as np

from cuda.compute import segmented_reduce

def sum_op(a, b):
    return a+b

awkward_array = ak.Array([[1], [2, 3], [4, 5], [6, 7, 8], [], [9]], backend = 'cuda')
input_data = awkward_array.layout.content.data
offsets = awkward_array.layout.offsets.data

# Prepare the start and end offsets
start_o = offsets[:-1]
end_o = offsets[1:]

# Prepare the output array
n_segments = start_o.size
output = cp.empty(n_segments, dtype=cp.int32)

# Initial value for the reduction
h_init = np.array([0], dtype=np.int32)

# Perform the segmented reduce
segmented_reduce(
    input_data, output, start_o, end_o, sum_op, h_init, n_segments
)

print(f"Segmented reduce result: {output.get()}")

Segmented reduce result: [ 1  5  9 21  0  9]


Success!

In [2]:
# Verify the result.
expected_output = cp.asarray([1, 5, 9, 21, 0, 9], dtype=output.dtype)
assert (output == expected_output).all()

### Compare time metrics between cuda-kernels and cccl

Using cuda-kernels:

In [32]:
import cupy as cp

cuda_kernel = """
extern "C" {
    __global__ void awkward_reduce_sum_a(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;

       if (thread_id < outlength) {
          toptr[thread_id] = 0;
       }
    }
}
extern "C" {
    __global__ void awkward_reduce_sum_b(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
       int stride = blockDim.x * gridDim.x;

       for (int i = thread_id; i < lenparents; i += stride) {
           atomicAdd(&toptr[parents[i]], fromptr[i]);
       }
    }
}
"""

In [4]:
%%timeit
parents = cp.array([0, 1, 1, 2, 2, 3, 3, 3, 5], dtype=cp.int32)
fromptr = cp.array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=cp.int32)
lenparents = len(parents)
outlength = int(cp.max(parents)) + 1
toptr = cp.zeros(outlength, dtype=cp.int32)

block_size = 256
grid_size = (lenparents + block_size - 1) // block_size

raw_module = cp.RawModule(code=cuda_kernel)

awkward_reduce_sum_a = raw_module.get_function('awkward_reduce_sum_a')
awkward_reduce_sum_b = raw_module.get_function('awkward_reduce_sum_b')

awkward_reduce_sum_a((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))
awkward_reduce_sum_b((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))

331 μs ± 10.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Using cccl:

In [105]:
%%timeit
awkward_array = rand_arr
input_data = awkward_array.layout.content.data 
offsets = awkward_array.layout.offsets.data

# Prepare the start and end offsets
start_o = offsets[:-1]
end_o = offsets[1:]

# Prepare the output array
n_segments = start_o.size
output = cp.empty(n_segments, dtype=np.int32)

# Initial value for the reduction
h_init = np.array([0], dtype=np.int32)

# Perform the segmented reduce
segmented_reduce(
    input_data, output, start_o, end_o, sum_op, h_init, n_segments
)

The slowest run took 7.03 times longer than the fastest. This could mean that an intermediate result is being cached.
138 μs ± 122 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Generating a big (2GB) array for testing

In [10]:
# randomly generate a ListOffsetArray
array_size = 10000000
# inner arrays lengths 
lengths = cp.random.randint(0, 6, array_size)
# a flat array of all the data
layout = ak.contents.NumpyArray(cp.random.random(int(sum(lengths))))
# calculate offsets from the lenghts
offsets = ak.index.Index(cp.concatenate((cp.array([0]), cp.cumsum(lengths))))

rand_arr = ak.Array(ak.contents.ListOffsetArray(offsets, layout))
rand_arr

<Array [[0.5446351700822585, ...], ..., [...]] type='10000000 * var * float64'>

Save the arrays to the file

In [33]:
ak.to_parquet(rand_arr, "random_listoffset_small.parquet")

  created_by: parquet-cpp-arrow version 22.0.0
  num_columns: 1
  num_rows: 10000000
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [8]:
ak.to_parquet(ak.concatenate([rand_arr] * 9), "random_listoffset.parquet")

  created_by: parquet-cpp-arrow version 22.0.0
  num_columns: 1
  num_rows: 90000000
  num_row_groups: 2
  format_version: 2.6
  serialized_size: 0

### Test cccl on bigger data

In [3]:
awkward_array = ak.to_backend(ak.from_parquet("random_listoffset_small.parquet"), 'cuda')

Let's modify our sum function for float64 values:

In [4]:
def cccl_sum(awkward_array):
    input_data = awkward_array.layout.content.data 
    offsets = awkward_array.layout.offsets.data
    
    # Prepare the start and end offsets
    start_o = offsets[:-1]
    end_o = offsets[1:]
    
    # Prepare the output array
    n_segments = start_o.size
    output = cp.empty(n_segments, dtype=cp.float64)
    
    # Initial value for the reduction
    h_init = np.array([0], dtype=np.float64)
    
    # Perform the segmented reduce
    segmented_reduce(
        input_data, output, start_o, end_o, sum_op, h_init, n_segments
    )
    return output

Check the results

In [6]:
cccl_sum(awkward_array)

array([2.03369381, 1.00149052, 0.74000209, ..., 2.11200709, 0.56703317,
       0.98825442], shape=(10000000,))

timeit the performance

In [7]:
%%timeit -r 7 -n 100
cccl_sum(awkward_array)

79.5 μs ± 18.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Load a 9times bigger array

In [2]:
awkward_array2 = ak.to_backend(ak.from_parquet("random_listoffset.parquet"), 'cuda')

prerun

In [5]:
cccl_sum(awkward_array2)

array([1.05136775, 0.        , 0.62357382, ..., 0.9605404 , 0.41038346,
       1.98615626], shape=(90000000,))

timeit

In [6]:
%%timeit -r 7 -n 100
cccl_sum(awkward_array2)

99.4 μs ± 59.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


It takes about twice as much time

### Test cuda-kernel on bigger data

In [5]:
awkward_array = ak.to_backend(ak.from_parquet("random_listoffset_small.parquet"), 'cuda')

In [6]:
def get_parents(awkward_array, array_data):
    offsets = awkward_array.layout.offsets.data

    # get the 'parents'
    # number of all items
    N = len(array_data)
    # all the possible parents idx
    flat_idx = cp.arange(N)
    # use searchsorted to map each flat item to its parent's index
    parents = cp.searchsorted(offsets[1:], flat_idx, side="right")
    return parents

In [7]:
import cupy as cp

cuda_kernel = """
extern "C" {
    __global__ void awkward_reduce_sum_a(double* toptr, double* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;

       if (thread_id < outlength) {
          toptr[thread_id] = 0.0f;
       }
    }
}
extern "C" {
    __global__ void awkward_reduce_sum_b(double* toptr, double* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
       int stride = blockDim.x * gridDim.x;

       for (int i = thread_id; i < lenparents; i += stride) {
           atomicAdd(&toptr[parents[i]], fromptr[i]);
       }       
    }
}
"""

Compare the results with cccl

In [9]:
def cuda_kernels_sum(awkward_array):
    fromptr = awkward_array.layout.content.data
    parents = get_parents(awkward_array, fromptr).astype(cp.int32)
    
    lenparents = len(parents)
    outlength = int(cp.max(parents)) + 1
    toptr = cp.zeros(outlength, dtype=cp.float64)
    
    block_size = 256
    grid_size = (lenparents + block_size - 1) // block_size
    
    raw_module = cp.RawModule(code=cuda_kernel)
    
    awkward_reduce_sum_a = raw_module.get_function('awkward_reduce_sum_a')
    awkward_reduce_sum_b = raw_module.get_function('awkward_reduce_sum_b')
    
    awkward_reduce_sum_a((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))
    awkward_reduce_sum_b((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))
    return toptr.get()

In [10]:
cuda_kernels_sum(awkward_array)

array([2.03369381, 1.00149052, 0.74000209, ..., 2.11200709, 0.56703317,
       0.98825442], shape=(10000000,))

Results are the same! Now let's see how long it takes to use cuda kernels

In [12]:
%%timeit -r 7 -n 100
cuda_kernels_sum(awkward_array)

29.2 ms ± 633 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


test on a 9times bigger array

In [13]:
awkward_array2 = ak.to_backend(ak.from_parquet("random_listoffset.parquet"), 'cuda')

prerun

In [14]:
cuda_kernels_sum(awkward_array2)

array([1.05136775, 0.        , 0.62357382, ..., 0.9605404 , 0.41038346,
       1.98615626], shape=(90000000,))

timeit

In [15]:
%%timeit -r 7 -n 100
cuda_kernels_sum(awkward_array2)

236 ms ± 4.87 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


This array(~2GB) takes roughly 7 times longer to process then a smaller one(280Mb). Comparing to cccl, it was only a two times longer for a bigger array.